In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/ml/Steel_industry_data.csv")

In [ ]:
df

target variable usage_kwh

In [ ]:
df.isnull().sum()

performming label encoding


In [ ]:
df['Day_of_week'].unique() #label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
categories = df['Day_of_week']
encoded_labels = label_encoder.fit_transform(categories)
df['Day_of_week']=encoded_labels
print(encoded_labels)

In [ ]:
df['WeekStatus'].unique() #label encoding

In [ ]:

label_encoder = LabelEncoder()
categories = df['WeekStatus']
encoded_labels = label_encoder.fit_transform(categories)
df['WeekStatus']=encoded_labels

print(encoded_labels)



In [ ]:
df['Load_Type'].unique() #ordinal encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
categories = df['Load_Type']
encoded_labels = ordinal_encoder.fit_transform(categories.values.reshape(-1, 1))
df['Load_Type']=encoded_labels

print(encoded_labels)


In [ ]:
df.info()

because in our dataset date and time is important we are converting it and make that model can understand

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour

df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

df.head()


In [ ]:
df['year'].unique()

In [ ]:
df.columns

In [ ]:
df.tail()

In [ ]:
df = df.drop(['year','date','WeekStatus'], axis=1,inplace=False)

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

outlier_lower = Q1 - 1.5 * IQR
outlier_upper = Q3 + 1.5 * IQR

outliers = (df < outlier_lower) | (df > outlier_upper)
print("Outlier counts per column:\n", outliers.sum())


In [ ]:
df = df[~((df < outlier_lower) | (df > outlier_upper)).any(axis=1)]
print("Shape after removing outliers:", df.shape)


In [ ]:
df.columns

In [ ]:

correlation_matrix = df.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Matrix Heatmap")
plt.show()


In [ ]:
print(correlation_matrix['Usage_kWh'][['NSM', 'hour']])
print(correlation_matrix[['NSM', 'hour']])


In [ ]:
df.drop(['NSM'], axis=1, inplace=True)

starting of model building
do->
1)train test split
2)standarad scalar
3)cross validation
4)model evaluation

In [ ]:
X = df.drop('Usage_kWh',axis=1)
y= df['Usage_kWh']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
#help(SVR)

In [ ]:
# Linear Kernel
svr_linear = SVR(kernel='linear')
svr_linear.fit(X_train, y_train)
# Evaluate Linear Kernel
y_pred_linear = svr_linear.predict(X_test)
r2_linear = r2_score(y_test, y_pred_linear)
mse_linear = mean_squared_error(y_test, y_pred_linear)
print(f"Linear Kernel R² Score: {r2_linear:.2f}")
print(f"Linear Kernel Mean Squared Error: {mse_linear:.2f}")


In [ ]:

# RBF Kernel
svr_rbf = SVR(kernel='rbf', C=1.0, gamma=0.1, epsilon=0.1)
svr_rbf.fit(X_train, y_train)

# Evaluate RBF Kernel
y_pred_rbf = svr_rbf.predict(X_test)
r2_rbf = r2_score(y_test, y_pred_rbf)
mse_rbf = mean_squared_error(y_test, y_pred_rbf)
print(f"RBF Kernel R² Score: {r2_rbf:.2f}")
print(f"RBF Kernel Mean Squared Error: {mse_rbf:.2f}")
